使用Mido第三方库，在展开实验前，先规定一些参数

In [41]:
import mido
import numpy as np
from random import randint, choice, random, seed
from typing import List, Literal, Union
from time import time
from copy import deepcopy
import math

BPM = 120  # Beats per minute
VELOCITY = 64  # MIDI note velocity
TICK_PER_BEAT = 480  # Tick per beat

# The rhythm of the generated music
NUMERATOR = 4
DENOMINATOR = 4  # 4/4

# The range of notes to be generated
NOTE_MIN = 60  # C4
NOTE_MAX = 84  # C6

# Note lengths
WHOLE = 4 * TICK_PER_BEAT
HALF = 2 * TICK_PER_BEAT
QUARTER = 1 * TICK_PER_BEAT
EIGHTH = QUARTER // 2
SIXTEENTH = QUARTER // 4
NOTE_LENGTH = [HALF, QUARTER, EIGHTH]

BAR_LENGTH = NUMERATOR * TICK_PER_BEAT

In [42]:
# interval -> value
# 0 fot unison, 12 for octave
# if interval > 12, the value will be 'large_interval_value'
interval_value_dict = {
    0: 1, 1: 2, 2: 2, 3: 2, 4: 2, 5: 1, 6: 3, 7: 1, 8: 3, 9: 3, 10: 4, 11: 4, 12: 3
}
large_interval_value = 5

alpha, beta, gamma = 1, 4, 0.5
mean_coeff = np.array([2, 1, 1, 1, 1, 1, 1, 2])
standard_coeff = np.array([1, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 1])
target = 2.0

mutation_rate = 0.8

音符的包装——包含起始时间、持续时间、音高、音量的属性，同时提供了音名翻译的接口。

In [43]:
note_name_dict = {
    'C': 0, 'C#': 1, 'Cb': -1,
    'D': 2, 'D#': 3, 'Db': 1,
    'E': 4, 'E#': 5, 'Eb': 3, 
    'F': 5, 'F#': 6, 'Fb': 4,
    'G': 7, 'G#': 8, 'Gb': 6, 
    'A': 9, 'A#': 10, 'Ab': 8,
    'B': 11, 'B#': 12, 'Bb': 10,
}

Key_Major_T = Literal['C', 'Db', 'D', 'Eb', 'E', 'F',
                      'F#', 'E', 'G', 'Ab', 'A', 'Bb', 'B']
Key_Minor_T = Literal['Cm', 'C#m', 'Dm', 'D#m', 'Ebm', 'Em', 
                      'Fm', 'F#m', 'Gm', 'G#m', 'Am', 'Bbm', 'Bm']
Key_T = Union[Key_Major_T, Key_Minor_T]
Pitch_T = int

# offset of the notes in the major mode and minor mode
major_offset = (0, 2, 4, 5, 7, 9, 11)
minor_offset = (0, 2, 3, 5, 7, 8, 10)


class Note:
    def __init__(self, pitch: Pitch_T, length: int, 
                 start_time: int, velocity: int = VELOCITY):
        # Here the "time" is "tick" in mido actually
        self.pitch = pitch
        self.length = length
        self.start_time = start_time
        self.velocity = velocity

    def __str__(self):
        pitch = self.pitch_to_name(self.pitch)
        return f"{pitch}: length={self.length} start={self.start_time} velocity={self.velocity}"

    @property
    def end_time(self):
        return self.start_time + self.length

    @staticmethod
    def in_mode(pitch: Pitch_T, key: Key_T):
        """judge if the note is in the given mode."""
        if key.endswith('m'):
            # minor mode
            base = note_name_dict[key[:-1]]
            return (pitch - base) % 12 in minor_offset
        else:
            # major mode
            base = note_name_dict[key]
            return (pitch - base) % 12 in major_offset
    
    @staticmethod
    def random_pitch_in_mode(key: Key_T):
        """Generate a random note in the given mode."""
        while True:
            pitch = randint(NOTE_MIN, NOTE_MAX)
            if Note.in_mode(pitch, key):
                return pitch

    @staticmethod    
    def ord_in_mode(key: Key_T, pitch: Pitch_T):
        """Get the order of the note in the given mode.
        For example, in C major, C is 1, D is 2, E is 3, etc."""
        if key.endswith('m'):
            # minor mode
            base = note_name_dict[key[:-1]]
            return minor_offset.index((pitch - base) % 12) + 1
        else:
            # major mode
            base = note_name_dict[key]
            return major_offset.index((pitch - base) % 12) + 1
            
    @staticmethod
    def name_to_pitch(note_name: str) -> Pitch_T:
        """Convert a note name to a pitch.
        For example, "C4" -> 60. """
        octave = note_name[-1]
        name = note_name[:-1]
        pitch = note_name_dict[name]
        return (int(octave) + 1) * 12 + pitch

    @staticmethod
    def pitch_to_name(pitch: Pitch_T) -> str:
        """Convert a pitch to a note name.
        For example, 60 -> "C4"."""
        octave = pitch // 12 - 1
        note = pitch % 12
        for name, picth in note_name_dict.items():
            if note == picth:
                return name + str(octave)

对音轨的包装
- `self.note`是所有音符构成的列表，直接访问即可
- 提供了与`mido.MidiTrack`相互转化的接口
- 完成了对音轨的移调、倒影、逆行工作

In [44]:

class Track:
    """A wrapper for mido.MidiTrack."""

    def __init__(self, instrument: int = 0, key: Key_T = 'C'):
        self.instrument = instrument
        self.key = key
        self.note: list[Note] = []

    def __str__(self):  # used for debug
        meta_msg = f"Key: {self.key}\nInstrument: {self.instrument}\n"
        bars = self.split_into_bars()
        note_msg = ''
        for idx, bar in enumerate(bars):
            note_msg += f'\n-------------------  Bar {idx + 1}\n'   
            note_msg += '\n'.join(str(note) for note in bar)
        return meta_msg + note_msg

    def print_brief_info(self):
        """Brief information of the track."""
        print(f"Key: {self.key}")
        print(f"Instrument: {self.instrument}")
        print(f"Length: {self.full_length}")
        print(f"Bar: {self.bar_number}")

    @staticmethod
    def from_track(track: mido.MidiTrack):
        """Generate a track from a midi track.
        Available for chords."""

        ga_track = Track()
        time = 0
        note_dict = {}  # pitch -> (start_time, velocity)
        for msg in track:
            if msg.type == 'program_change':
                ga_track.instrument = msg.program
            elif msg.type == 'key_signature':
                ga_track.key = msg.key
            elif msg.type == 'note_on':
                time += msg.time
                note_dict[msg.note] = (time, msg.velocity)
            elif msg.type == 'note_off':
                time += msg.time
                start_time, velocity = note_dict.pop(msg.note)
                ga_track.note.append(
                    Note(msg.note, time - start_time, start_time, velocity))
        return ga_track

    def to_track(self) -> mido.MidiTrack:
        """Generate a midi track from a track.
        Available for chords."""

        midi_track = mido.MidiTrack()
        midi_track.append(mido.MetaMessage(
            'key_signature', key=self.key, time=0))
        midi_track.append(mido.Message(
            'program_change', program=self.instrument, time=0))

        event_set = set()  # (time, 0/1(note_off/on), note)
        for note in self.note:
            event_set.add((note.start_time, 1, note))
            event_set.add((note.start_time + note.length, 0, note))
        # Sort all the events by time. If the time is same, note_off is before note_on.
        sorted_time = sorted(event_set, key=lambda x: (x[0], x[1]))

        last_time = 0
        for time, event_num, note in sorted_time:
            event = 'note_on' if event_num else 'note_off'
            msg = mido.Message(
                event, note=note.pitch, velocity=note.velocity, time=time-last_time)
            midi_track.append(msg)
            last_time = time
        return midi_track

    def generate_random_pitch_on_rhythm(self, track: 'Track'):
        """Generate random pitches on the given track with rhythm."""
        for note in track.note:
            note.pitch = Note.random_pitch_in_mode(self.key)
        # We want the pitch of the last note is the tonic
        while True:
            pitch = Note.random_pitch_in_mode(self.key)
            if Note.ord_in_mode(self.key, pitch) == 1:
                track.note[-1].pitch = pitch
                return track

    def generate_random_track(self, bar: int):
        """Generate a random track with the given bar number"""
        for i in range(bar - 1):
            length = BAR_LENGTH
            while length > 0:
                l = choice(NOTE_LENGTH)
                if l <= length:
                    note = Note(0, l, (i + 1) * BAR_LENGTH - length)
                    length -= l
                    self.note.append(note)

        # For the last bar, we want to make sure that the last note is a half note
        length = BAR_LENGTH
        while length > HALF:
            l = choice(NOTE_LENGTH)
            if l <= length - HALF:
                note = Note(0, l, bar * BAR_LENGTH - length)
                length -= l
                self.note.append(note)
        note = Note(0, HALF, bar * BAR_LENGTH - HALF)
        self.note.append(note)
        self.generate_random_pitch_on_rhythm(self)
        return self

    def split_into_bars(self) -> List[List[Note]]:
        """Split the track into bars."""
        bars = [[] for _ in range(self.bar_number)]
        for note in self.note:
            idx = note.start_time // BAR_LENGTH

            if note.start_time + note.length <= (idx + 1) * BAR_LENGTH:
                bars[idx].append(note)
            else:
                # The note exceeds the bar, split it into two parts
                note1, note2 = deepcopy(note), deepcopy(note)
                bar_time = (idx + 1) * BAR_LENGTH
                note1.length = bar_time - note.start_time
                note2.length = note.start_time + note.length - bar_time
                note2.start_time = bar_time
                bars[idx].append(note1)
                bars[idx + 1].append(note2)
        return bars

    def join_bars(self, bars: List[List[Note]]):
        """Join the bars into a track."""
        self.note = [note for bar in bars for note in bar]
        return self

    @property
    def full_length(self):
        """The length of the track"""
        return max(note.start_time + note.length for note in self.note)

    @property
    def bar_number(self):
        """The number of bars"""
        return math.ceil(self.full_length / WHOLE)

    def transpose(self, interval):
        """Transpose the track by the given interval"""
        for note in self.note:
            note.pitch += interval

    def inverse(self, center):
        """Inverse the track by the given center"""
        for note in self.note:
            note.pitch = 2 * center - note.pitch

    def retrograde(self):
        """Retrograde the track"""
        for note in self.note:
            note.start_time = self.full_length - note.end_time
        self.note.reverse()

测试函数
- `get_midi` 生成一个给定调式的空白midi，同时建立好节拍、曲速、调式的元信息
- `generate_random_midi` 生成一个随机midi，调式指定为#g小调
- `read_midi` 读取一个现有的midi——久石让的《Summer》间奏片段。其中包含和弦、音符强弱变化以用于测试

In [45]:
def get_midi(key: Key_T = None):
    s = mido.MidiFile()
    meta_track = mido.MidiTrack()
    meta_track.append(mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(BPM)))
    meta_track.append(mido.MetaMessage(
        'time_signature', numerator=NUMERATOR, denominator=DENOMINATOR))
    if key is not None:
        meta_track.append(mido.MetaMessage(
            'key_signature', key=key, time=0))
    s.tracks.append(meta_track)
    return s

In [46]:
def generate_random_midi_test():
    s = get_midi()
    track = Track(0, 'G#m')
    track.generate_random_track(4)  # 4 bars
    s.tracks.append(track.to_track())
    retrograde_track = deepcopy(track)  # MUST use deepcopy
    retrograde_track.retrograde()
    s.tracks.append(retrograde_track.to_track())
    s.save('random.mid')

    # Expected result:
    # random.mid: A random piece with 4 bars in G sharp minor which has 2 tracks,
    # one is the original track, the other is the retrograde track.


def read_midi_test():
    s = mido.MidiFile('test.mid')
    right_hand = Track.from_track(s.tracks[0])
    right_hand.print_brief_info()
    print('----------------')
    left_hand = Track.from_track(s.tracks[1])
    left_hand.print_brief_info()
    # For more information, use print(track) to see the detailed notes.
    print(left_hand.bar_number)

    # Expected output:
    # Key: D
    # Instrument: 0
    # Length: 15353
    # Bar: 8
    # ----------------
    # Key: D
    # Instrument: 0
    # Length: 15347
    # Bar: 8


generate_random_midi_test()
read_midi_test()

Key: D
Instrument: 0
Length: 15353
Bar: 8
----------------
Key: D
Instrument: 0
Length: 15347
Bar: 8
8


接下来做遗传算法的准备工作

In [47]:

class TrackParameter:
    """Used to calculate parameters of the tracks"""

    def __init__(self, track: Track) -> None:
        self.track = track
        self.bar_number = track.bar_number
        self.bars = self.track.split_into_bars()

        # pitch parameters
        self.means = np.zeros(self.bar_number, dtype=float)
        self.standard = np.zeros(self.bar_number, dtype=float)
        self.bad_notes = 0
        # rhythm parameters
        self.strong_beats = 0
        self.weak_beats = 0
        self.echo = 0

    @staticmethod
    def _interval_to_value(interval: int):
        return interval_value_dict.get(abs(interval), large_interval_value)

    def update_pitch_parameters(self):
        self._update_interval_parameters()
        self._update_bad_notes()
        
    def update_rhythm_parameters(self):
        self._update_beats()
        self._update_echo()

    def _update_interval_parameters(self):
        for idx, bar in enumerate(self.bars):
            values = []
            for i in range(len(bar) - 1):
                interval = bar[i + 1].pitch - bar[i].pitch
                values.append(self._interval_to_value(interval))
            if idx > 0:
                # add the interval between the last note of the previous bar
                # and the first note of the current bar
                if bar == []: 
                    raise Exception("Empty bar")
                if self.bars[idx - 1] == []:
                    raise Exception("Empty bar")
                interval = bar[0].pitch - self.bars[idx - 1][-1].pitch
                values.append(self._interval_to_value(interval))

            if not values:
                self.means[idx] = 0
                self.standard[idx] = 0
                continue

            self.means[idx] = np.mean(values)
            self.standard[idx] = np.std(values)

    def _update_bad_notes(self):
        self.bad_notes = 0
        for note in self.track.note:
            if not Note.in_mode(note.pitch, self.track.key):
                self.bad_notes += 1

    def _update_beats(self):
        self.strong_beats = 0
        for bar in self.bars:
            for note in bar:
                if note.start_time % HALF == 0:
                    self.strong_beats += 1
                elif note.start_time % QUARTER == 0:
                    self.weak_beats += 1

    def _update_echo(self):
        # Bar 0 and 2; 1 and 3; 4 and 6; 5 and 7 are echo, etc.
        # If they have the similar rhythm, the echo will be higher
        for bar in range(0, self.bar_number, 4):
            self.echo += self._rhythm_similarity_of_bars(
                self.bars[bar], self.bars[bar + 2])
            self.echo += self._rhythm_similarity_of_bars(
                self.bars[bar + 1], self.bars[bar + 3])
        
    @staticmethod
    def _rhythm_similarity_of_bars(bar1: List[Note], bar2: List[Note]):
        """Calculate the similarity of the rhythm of two bars.
        The similarity is the number of notes in the same position."""
        same = 0
        for note1 in bar1:
            for note2 in bar2:
                if (note1.start_time - note2.start_time) % BAR_LENGTH == 0:
                    same += 1
        return (same ** 2) / (len(bar1) * len(bar2))

In [48]:
class GAForPitch:

    def __init__(self,
                 reference_track: Track,
                 population: List[Track],
                 mutation_rate: float
                 ):

        self.ref_track = reference_track
        self.ref_param = TrackParameter(self.ref_track)
        self.ref_param.update_pitch_parameters()
        self.population = population
        self.mutation_rate = mutation_rate
        self.fitness = [0] * len(population)
        self._update_fitness()
        self.best_index, self.second_index = 0, 0

    def _update_fitness(self):
        for idx, track in enumerate(self.population):
            self.fitness[idx] = self._get_fitness(track)

    def _get_fitness(self, track: Track) -> float:
        # It's better to have a lower fitness
        track_param = TrackParameter(track)
        track_param.update_pitch_parameters()
        mean_diff = np.abs(track_param.means - self.ref_param.means)
        f1 = alpha * np.dot(mean_coeff, mean_diff)
        standard_diff = np.abs(track_param.standard - self.ref_param.standard)
        f2 = beta * np.dot(standard_coeff, standard_diff)
        g = gamma * TrackParameter(track).bad_notes
        return f1 + f2 + g

    def select(self):
        self._update_fitness()
        for i in range(len(self.fitness)):
            if self.fitness[i] < self.fitness[self.best_index]:
                self.best_index = i
                self.second_index = self.best_index
            elif self.fitness[i] < self.fitness[self.second_index]:
                self.second_index = i

    def crossover(self):
        # for i in range(len(self.population)):
        #     index1 = self.best_index if randint(0,1) else self.second_index
        #     index2 = self.best_index if randint(0,1) else self.second_index
        #     bars1 = deepcopy(self.population[index1]).split_into_bars()
        #     bars2 = deepcopy(self.population[index2]).split_into_bars()
        #     cross_point = randint(0, self.ref_param.bar_number - 1)
        #     bars = bars1[:cross_point] + bars2[cross_point:]
        #     self.population[i] = self.population[i].join_bars(bars)
        # self._update_fitness()
        pass

    def mutate(self):
        for i in range(len(self.population)):
            if random() > self.mutation_rate:
                continue
            # TODO: mutation
            track = deepcopy(self.population[choice(
                [self.best_index, self.second_index])])
            mutate_type = randint(1, 3)
            # When mutating, do not change the last note pitch,
            # because we want the last note to be the tonic
            if mutate_type == 1:
                self._mutate_1(track)
            elif mutate_type == 2:
                self._mutate_2(track)
            else:
                self._mutate_3(track)
            self.population[i] = track

    @staticmethod
    def _mutate_1(track: Track):
        # If the interval between two notes is too large, change it
        for idx in range(len(track.note) - 1):
            if track.note[idx + 1].pitch - track.note[idx].pitch > 12:
                track.note[idx].pitch += 12
            elif track.note[idx + 1].pitch - track.note[idx].pitch < -12:
                track.note[idx].pitch -= 12

    @staticmethod
    def _mutate_2(track: Track):
        # Change the pitch of a random note
        idx = randint(0, len(track.note) - 2)
        track.note[idx].pitch = Note.random_pitch_in_mode(track.key)

    @staticmethod
    def _mutate_3(track: Track):
        # Swap two notes' pitch
        idx = randint(1, len(track.note) - 2)
        track.note[idx], track.note[idx - 1] = \
            track.note[idx - 1], track.note[idx]

    def show_info(self):
        print("Now the best fitness for pitch is", self.fitness[self.best_index])

    def run(self, generation):
        print("Start training for pitch...")
        for i in range(generation):
            print(f"Generation {i}:")
            self.select()
            self.crossover()
            self.mutate()
            self.show_info()
            if self.fitness[self.best_index] < target:
                print(f"[!] Target reached at generation {i}")
                print(f"final fitness for pitch: {self.fitness[self.best_index]}")
                return self.population[self.best_index]

        print(f"[!] Target not reached after {generation} generations")
        print(f"final fitness for pitch: {self.fitness[self.best_index]}")
        return self.population[self.best_index]

In [49]:
theta = 0.5
delta = 1
target2 = 3

class GAForRhythm:
    
    def __init__(self,
                 population: List[Track],
                 mutation_rate: float
                 ):
        self.population = population
        self.bar_number = population[0].bar_number
        self.mutation_rate = mutation_rate
        self.fitness = [0] * len(population)
        self._update_fitness()
        self.best_index, self.second_index = 0, 0

    def _update_fitness(self):
        for idx, track in enumerate(self.population):
            self.fitness[idx] = self._get_fitness(track)

    def _get_fitness(self, track: Track) -> float:
        # It's better to have a higher fitness
        fitness = 0
        # give punishment if the number of strong beats is not enough
        param = TrackParameter(track)
        param.update_rhythm_parameters()
        fitness += (param.strong_beats - 2 * self.bar_number) * theta
        # give encouragement if echo is high
        fitness += param.echo * delta
        print( (param.strong_beats - 2 * self.bar_number) * theta, param.echo * delta)
        return fitness 
    
    def select(self):
        self._update_fitness()
        for i in range(len(self.fitness)):
            if self.fitness[i] > self.fitness[self.best_index]:
                self.best_index = i
                self.second_index = self.best_index
            elif self.fitness[i] > self.fitness[self.second_index]:
                self.second_index = i

    def crossover(self):
        for i in range(len(self.population)):
            index1 = self.best_index if randint(0,1) else self.second_index
            index2 = self.best_index if randint(0,1) else self.second_index
            bars1 = deepcopy(self.population[index1]).split_into_bars()
            bars2 = deepcopy(self.population[index2]).split_into_bars()
            cross_point = randint(0, self.bar_number // 2 - 1) * 2
            bars = bars1[:cross_point] + bars2[cross_point:]
            self.population[i] = self.population[i].join_bars(bars)

    def mutate(self):
        for i in range(len(self.population)):
            if random() > self.mutation_rate:
                continue
            # TODO: mutation
            track = deepcopy(self.population[choice(
                [self.best_index, self.second_index])])
            mutate_type = randint(1, 4)
            # When mutating, do not change the last note pitch,
            # because we want the last note to be the tonic
            if mutate_type == 1:
                self._mutate_1(track)
            elif mutate_type == 2:
                self._mutate_2(track)
            elif mutate_type == 3:
                self._mutate_3(track)
            else:
                self._mutate_4(track)
            self.population[i] = track

    @staticmethod
    def _mutate_1(track: Track):
        # Swap two notes' length
        idx = randint(0, len(track.note) - 2)
        note1, note2 = track.note[idx], track.note[idx + 1]
        if note2.start_time // BAR_LENGTH != note1.start_time // BAR_LENGTH:
            # The two notes are in different bars, we can't swap them
            return
        end = note2.end_time
        note1.length, note2.length = note2.length, note1.length
        note2.start_time = end - note2.length
            
    @staticmethod
    def _mutate_2(track: Track):
        # Split a note into two notes
        idx = randint(0, len(track.note) - 2)
        note = track.note[idx]
        if note.length == EIGHTH:  # We can't split it
            return
        while True:
            length = choice(NOTE_LENGTH)
            if length < note.length:
                end = note.end_time
                note.length -= length
                new_note = Note(note.pitch, length, end - length, note.velocity)
                track.note.insert(idx + 1, new_note)
                return
                
    @staticmethod
    def _mutate_3(track: Track):
        # merge two notes into one note
        idx = randint(0, len(track.note) - 3)
        note = track.note[idx]
        if track.note[idx + 1].start_time % BAR_LENGTH == 0:
            # The next note is at the beginning of a bar, we can't merge it
            return
        note.length = track.note[idx + 1].end_time - note.start_time
        track.note.pop(idx + 1)
        
    @staticmethod
    def _mutate_4(track: Track):
        # copy a bar and paste it to another bar
        idx = randint(2, track.bar_number - 1)
        bars = track.split_into_bars()
        bars[idx - 2] = deepcopy(bars[idx])
        for note in bars[idx - 2]:
            note.start_time -= BAR_LENGTH * 2
        track.join_bars(bars)
        
    def show_info(self):
        print("Now the best fitness for rhythm is", self.fitness[self.best_index])

    def run(self, generation):
        print("Start training for rhythm...")
        for i in range(generation):
            print(f"Generation {i}:")
            self.select()
            self.crossover()
            self.mutate()
            self.show_info()
            if self.fitness[self.best_index] > target2:
                print(f"[!] Target reached at generation {i}")
                print(f"final fitness for rhythm: {self.fitness[self.best_index]}")
                return self.population[self.best_index]

        print(f"[!] Target not reached after {generation} generations")
        print(f"final fitness for rhythm: {self.fitness[self.best_index]}")
        return self.population[self.best_index]

遗传算法的运行测试

In [50]:
KEY = 'Db'
t0 = time()

ref_midi = mido.MidiFile('reference.mid')
ref_track = Track.from_track(ref_midi.tracks[0])

population = [Track(0, KEY).generate_random_track(8) for _ in range(10)]
ga = GAForRhythm(population, mutation_rate)
rhythm_track = ga.run(1000)

# print(rhythm_track)

# Use the rhythm of the track below
population2 = [Track(0, KEY).generate_random_pitch_on_rhythm(rhythm_track) for _ in range(10)]
ga2 = GAForPitch(ref_track, population2, mutation_rate)
result = ga2.run(1000)

s = get_midi(KEY)
s.tracks.append(result.to_track())
left_hand = Track.from_track(ref_midi.tracks[1])
for note in left_hand.note:
    note.velocity = VELOCITY
s.tracks.append(left_hand.to_track())

s.save('result.mid')
print(f"Time cost: {time() - t0}s")

-1.0 1.8111111111111111
-2.0 1.7166666666666666
-2.0 1.4375
-1.0 1.6291666666666667
-1.5 1.5277777777777777
-1.5 1.5
-2.0 2.1916666666666664
-2.0 1.5058333333333331
-2.0 1.2791666666666668
-2.0 1.0833333333333335
Start training for rhythm...
Generation 0:
-1.0 1.8111111111111111
-2.0 1.7166666666666666
-2.0 1.4375
-1.0 1.6291666666666667
-1.5 1.5277777777777777
-1.5 1.5
-2.0 2.1916666666666664
-2.0 1.5058333333333331
-2.0 1.2791666666666668
-2.0 1.0833333333333335
Now the best fitness for rhythm is 0.8111111111111111
Generation 1:
-1.0 1.8111111111111111
-1.0 1.8111111111111111
-1.0 1.8111111111111111
-1.5 1.4777777777777776
-1.0 2.033333333333333
-1.0 1.6777777777777778
-1.0 1.8111111111111111
-1.5 2.3666666666666667
-1.0 2.1444444444444444
-1.0 1.8111111111111111
Now the best fitness for rhythm is 1.1444444444444444
Generation 2:
-1.0 2.1444444444444444
-1.0 2.0444444444444447
-1.0 2.3111111111111113
-1.5 2.7
-1.0 2.1444444444444444
-1.5 1.5888888888888888
-1.0 1.8666666666666665
-1.

Now the best fitness for rhythm is 3.0
Generation 27:
-1.0 3.75
-1.0 3.75
-1.0 4.0
-1.0 4.0
-1.0 3.55
-0.5 3.1944444444444446
-1.0 3.75
-1.0 4.0
-1.0 3.55
-0.5 3.1944444444444446
Now the best fitness for rhythm is 3.0
Generation 28:
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 3.75
-1.0 4.0
-1.0 4.0
-1.0 4.0
Now the best fitness for rhythm is 3.0
Generation 29:
-1.5 3.4444444444444446
-1.0 3.8
-1.0 3.75
-1.0 3.75
-1.5 3.39
-1.0 3.75
-1.5 3.4444444444444446
-1.0 4.0
-1.0 3.75
-1.5 3.3333333333333335
Now the best fitness for rhythm is 3.0
Generation 30:
-1.0 3.75
-1.0 3.75
-1.0 4.0
-1.0 3.75
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 3.333333333333333
-1.0 3.333333333333333
-1.0 3.5625
Now the best fitness for rhythm is 3.0
Generation 31:
-1.5 3.4444444444444446
-1.0 3.75
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.5 3.75
-1.0 3.8
-1.0 4.0
Now the best fitness for rhythm is 3.0
Generation 32:
-1.0 4.0
-1.0 3.8
-1.0 4.0
-1.0 4.0
-1.0 4.0
-1.0 4.0
-0.5 3.2666666666666666
-1.5 3.4444

# TODO
- 关于节奏的优化
- 强拍音的和弦色彩